In [2]:
from estnltk import Text
import json
import os
from collections import defaultdict
import string 
import nltk
from collections import Counter
import re

In [10]:
# Sisendiks faili nimi ja praegune kaust ning sihtkohtade kaustad
def morphanalysis_and_save(filename, source, guess_destination, no_guess_destination, meta_destination):
    
    global keskmiste_arvutamiseks
    
    with open(os.path.join(source, filename), "r", encoding="UTF-8") as f:

        faili_keskmised = defaultdict(int)
        
        # Loeb failist vaid sisu, ignoreerib alguses olevat metainfot
        pure = "".join(f.readlines()[1:])
        
        # Regexiga leiab kõik potentsiaalsed emojid tekstist (kahe kooloni vaheline whitespaceita tekst)
        emojid = re.findall(":\S+?:", pure)
        # Eemaldab leitud emojide hulgast väärvasted
        wrong = [":http:", ":https:"]
        for value in wrong:
            while value in emojid:
                emojid.remove(value)
        # Eemaldab emojid tekstist, et nende sisu ei peetaks sõnadeks 
        for emoji in emojid:
            pure = pure.replace(emoji, "")

        # Otsib tekstist emotikone ja paneb need nimekirja
        emotikonid_leitud = []
        for emotikon in emotikonid:
            emotikonid_leitud.extend(re.findall(re.escape(emotikon), pure, re.IGNORECASE))
            # Eemaldab tekstist leitud emotikonid, et need sõnestamisel lahku löömisel need keskmiseid ei mõjutaks
            pure = re.sub(re.escape(emotikon), '', pure, re.IGNORECASE)
        # Vaatab tekstist eraldi emotikone, mis võivad olla ka kokkukleepumise tulemusel väärpositiivsed vasted
        # Lisaks eemaldad leitud emotikonid
        sobivad = []
        for emotikon in emotikonid_probleemsed:
            # Kui "silmad" on viimane emotikoni osa, kontrollib, et emotikoni ees ei oleks tegu tähemärgiga ehk et poleks seoses sõnaga
            if emotikon[-1] == ":":
                sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall("\W"+re.escape(emotikon), pure, re.IGNORECASE)), re.IGNORECASE))
                pure = re.sub("(\W)"+re.escape(emotikon), '\1', pure, re.IGNORECASE)
            # Vastasel juhul kontrollib seda emotikoni lõpust
            else:
                sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall(re.escape(emotikon)+"\W", pure, re.IGNORECASE)), re.IGNORECASE))
                pure = re.sub(re.escape(emotikon)+"(\W)", '\1', pure, re.IGNORECASE)
        
        # Jätab meelde emotikonide arvud ja loendid vigade kontrollimiseks
        faili_keskmised['emotikonide_arv']=len(emojid) + len(emotikonid_leitud) + len(sobivad)
        faili_keskmised['emojid_loend']=emojid
        faili_keskmised['emotikonid_loend']=emotikonid_leitud
        faili_keskmised['emotikonid_erilised_loend']=sobivad
        
        # Teeb morfoloogilise analüüsi nii tundmatude analüüsi oletamisega ja oletamiseta
        oletamisega = Text(pure)
        oletamiseta = Text(pure, disambiguate=False, guess=False, propername=False)

        oletamisega.tag_analysis()
        oletamiseta.tag_analysis()
        
        # Loob salvestamiseks failinimed
        ga_name = ".".join(filename.split(".")[:-1]) + "_morf_oletamisega.json"
        ta_name = ".".join(filename.split(".")[:-1]) + "_morf_oletamiseta.json"
        meta_name = ".".join(filename.split(".")[:-1]) + "_meta.json"
        # Salvest EstNLTK objektid failidesse
        with open(os.path.join(guess_destination, ga_name), 'w', encoding="UTF-8") as fp:
            json.dump(oletamisega, fp, sort_keys=True, indent=4)
        with open(os.path.join(no_guess_destination, ta_name), 'w', encoding="UTF-8") as fp:
            json.dump(oletamiseta, fp, sort_keys=True, indent=4)
        
        # Loeb kokku lemmade arvud ja käänduvate lemmade arvud
        kõikide_lemmade_arv = 0
        ainult_käänduvate_lemmade_arv = 0
            
        for lemma, postag in zip(oletamisega.lemmas, oletamisega.postags):
            kõikide_lemmade_arv += 1
            keskmiste_arvutamiseks['kõikide_lemmade_arv'] += 1
            # Kui tegu on käänduva lemmaga
            if postag in ["A", "C", "G", "H", "K", "N", "O", "S", "U", "Y"]:
                ainult_käänduvate_lemmade_arv += 1
                keskmiste_arvutamiseks['ainult_käänduvate_lemmade_arv'] += 1

        # Loeb kokku lemmad lahutades liitsõnad osasõnadeks
        lemmas_subwords = []
        for tokens in oletamisega.root_tokens:
            # Kui tegu on sõnega ja mitte loendiga
            if isinstance(tokens[0], str):
                for token in tokens:
                    # Kui kõik tähemärgid ei ole punktuatsioonimärgid
                    if not all(char in string.punctuation for char in token):
                        lemmas_subwords.append(token)
            # Kui tegu on loendiga, võtab esimese tõlgenduse
            else:
                for token in tokens[0]:
                    # Kui kõik tähemärgid ei ole punktuatsioonimärgid
                    if not all(char in string.punctuation for char in token):
                        lemmas_subwords.append(token)
                        
        # Jätab meelde osasõnade pikkused ja arvu
        keskmiste_arvutamiseks['lemmade_osasõnade_pikkused'] += sum(map(len, lemmas_subwords))
        keskmiste_arvutamiseks['lemmade_osasõnade_arv'] += len(lemmas_subwords)
        
        # Võtab sõnade algvormid, ignoreerib kirjavahemärke
        lemmad = [lemma.split("|")[0] for lemma in oletamisega.lemmas if not all(char in string.punctuation for char in lemma)]
        
        # Arvutab TTR-i, keskmise lemma osasõna pikkuse ja käänduvate lemmade osaarvu
        faili_keskmised['TTR'] = len(Counter(lemmad))/len(lemmad)
        faili_keskmised['keskmine_lemma_pikkus'] = sum(map(len, lemmas_subwords))/len(lemmas_subwords)
        faili_keskmised['käänduvate_lemmade_osaarv'] = ainult_käänduvate_lemmade_arv/kõikide_lemmade_arv
        
        # Liidab TTR-id üle failide kokku, et leida tekstide keskmine TTR
        keskmiste_arvutamiseks['TTR_keskmine'] += faili_keskmised['TTR']
        
        # Salvestab faili keskmised ja emotikonide info faili
        with open(os.path.join(meta_destination, meta_name), 'w', encoding="UTF-8") as fp:
            json.dump(faili_keskmised, fp, sort_keys=True, indent=4)

In [5]:
os.chdir("Downloads/Bakatöö/Gailit-2021")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Downloads/Bakatöö/Gailit-2021'

In [6]:
os.dir

AttributeError: module 'os' has no attribute 'dir'

In [1]:
# Loeb emotikonid sisse
emotikonid = []

for failinimi in ["wikipedia_emoticons_list.txt", "Unicode_emoticons_list.txt", "looks.wtf.txt", "unicode_emojis.txt"]:
    with open("Loendid/emotikonid/"+failinimi, "r", encoding="UTF-8") as fr:
        for line in fr.readlines():
            # Väiketähestab
            emotikonid.append(line.strip().lower())
# Eemaldab korduvad emotikonid
emotikonid = list(set(emotikonid))

# Mõned emotikonid võivad olla ka kokkukleepumise tõttu olla väärpositiivsed (":pole")
emotikonid_probleemsed = []
with open("Loendid/emotikonid/wikipedia_emoticons_sp.txt", "r", encoding="UTF-8") as fr:
    for line in fr.readlines():
        # Väiketähestab
        emotikonid_probleemsed.append(line.strip().lower())
# Eemaldab korduvad emotikonid 
emotikonid_probleemsed = list(set(emotikonid_probleemsed))

FileNotFoundError: [Errno 2] No such file or directory: 'Loendid/emotikonid/wikipedia_emoticons_list.txt'

In [8]:
# Kaustade nimed
source = "etnc19_web_2019_100000/"
target1 = "etnc19_web_2019_morf_oletamisega/"
target2 = "etnc19_web_2019_morf_oletamiseta/"
target3 = "etnc19_web_2019_meta/"

# Loob kaustad
os.makedirs(os.path.dirname(target1), exist_ok=True)
os.makedirs(os.path.dirname(target2), exist_ok=True)
os.makedirs(os.path.dirname(target3), exist_ok=True)

# Loob sõnastiku keskmiste arvutamiseks
keskmiste_arvutamiseks = defaultdict(int)

# Avab järjest kõik failid algkaustas
for file in [f for f in os.listdir(source)]:
    morphanalysis_and_save(file, source, target1, target2, target3)
    
# Loob keskmiste sõnastiku
keskmised = defaultdict(float)

# Arvutab keskmised
keskmised['TTR'] = keskmiste_arvutamiseks['TTR_keskmine']/100000
keskmised['keskmine_lemma_pikkus'] = keskmiste_arvutamiseks['lemmade_osasõnade_pikkused']/keskmiste_arvutamiseks['lemmade_osasõnade_arv']
keskmised['käänduvate_lemmade_osaarv'] = keskmiste_arvutamiseks['ainult_käänduvate_lemmade_arv']/keskmiste_arvutamiseks['kõikide_lemmade_arv']

# Salvestab keskmised faili
with open("keskmised.json", "w", encoding = "utf-8") as fw: 
    json.dump(keskmised, fw, sort_keys=True, indent=4)

Vana info

In [57]:
#Andmed
print('Keskmine sõnapikkus (liitsõnad osadena): {}'.format(avg_lemma_len))
print('Type-Token ratio                       : {}'.format(TTR))
print('Käändsõna osakaal                      : {}'.format(kaanduv_suhe_koik))
#print('Tajuverbide osakaal verbidest          : {}'.format(tajuverbid_suhe_all_verbs)) # Vaja alles üksikteksti analüüsiks

Keskmine sõnapikkus (liitsõnad osadena): 4.27369826435247
Type-Token ratio                       : 0.4489510489510489
Käändsõna osakaal                      : 0.2987951807228916
